# Usermore soap case study

Versão 1.1 Guilherme Fernandes 15:55 10/maio/2025 

In [1]:
from pprint import pprint

import gurobipy
from gurobipy import Model, GRB

import pandas as pd

## 1. Defining the sets

In [2]:
plants = [
    "Covington, KY",
    "New York, NY",
    "Arlington, TX",
    "Long Beach, CA",
]

existing_warehouses = [
    "Atlanta",
    "Boston",
    "Buffalo",
    "Chicago",
    "Cleveland",
    "Davenport",
    "Detroit",
    "Grand Rapids",
    "Greensboro",
    "Kansas City",
    "Baltimore",
    "Memphis",
    "Milwaukee",
    "Orlando",
    "Pittsburgh",
    "Portland",
    "W Sacramento",
    "W Chester",
]

potential_warehouses = [
    "Albuquerque",
    "Billings",
    "Denver",
    "El Paso",
    "Camp Hill",
    "Houston",
    "Las Vegas",
    "Minneapolis",
    "New Orleans",
    "Phoenix",
    "Richmond",
    "St Louis",
    "Salt Lake City",
    "San Antonio",
    "Seattle",
    "Spokane",
    "San Francisco",
    "Indianapolis",
    "Louisville",
    "Columbus",
    "New York",
    "Hartford",
    "Miami",
    "Mobile",
    "Memphis *",
    "Chicago *",
]

# Merge existing + potential
warehouses = existing_warehouses + potential_warehouses

print("Number of plants: ", len(plants))
print("Number of existing warehouses: ", len(existing_warehouses))
print("Number of potential warehouses: ", len(potential_warehouses))
print("Number of total warehouses: ", len(warehouses))
print("Obs.: Each plant is also an existing warehouse")

Number of plants:  4
Number of existing warehouses:  18
Number of potential warehouses:  26
Number of total warehouses:  44
Obs.: Each plant is also an existing warehouse


## 2. Parameters

In [3]:
# read state demands from Figure 1
state_demand = {
    # West region:
    "WA": 32437,  # Washington
    "OR": 31365,  # Oregon
    "CA": 135_116,  # California
    "NV": 16755,  # Nevada
    "AZ": 9063,  # Arizona
    "ID": 7153,  # Idaho
    # Northwest region:
    "UT": 9001,  # Utah
    "MT": 4140,  # Montana
    "ND": 5703,  # North Dakota
    "WY": 1004,  # Wyoming
    "CO": 11147,  # Colorado
    "SD": 1049,  # South Dakota
    "NE": 7347,  # Nebraska
    "KS": 6961,  # Kansas
    "MN": 5633,  # Minnesota
    "IA": 32175,  # Iowa
    "MO": 41680,  # Missouri
    # Southwest region:
    "NM": 3536,  # New Mexico
    "TX": 80438,  # Texas
    "OK": 13517,  # Oklahoma
    "AR": 4910,  # Arkansas
    "LA": 15011,  # Louisiana
    # Midwest region:
    "WI": 37448,  # Wisconsin
    "IL": 72839,  # Illinois
    "MI": 105_181,  # Michigan
    "IN": 43994,  # Indiana
    "KY": 3870,  # Kentucky
    "OH": 155_123,  # Ohio
    # Northeast region:
    "ME": 15829,  # Maine
    "NH": 4546,  # New Hampshire
    "RI": 17000,  # Rhode Island
    "NJ": 21154,  # New Jersey
    "NY": 160_917,  # New York
    "PA": 65108,  # Pennsylvania
    "CT": 26_187,  # Connecticut
    "MA": 37087,  # Massachusetts
    "VA": 17667,  # Virginia
    "WV": 9168,  # West Virginia
    "MD": 19284,  # Maryland
    "VT": 2928,  # Vermont
    "DE": 3044,  # Delaware
    # Southeast region:
    "TN": 42479,  # Tennessee
    "MS": 15_205,  # Mississippi
    "AL": 15835,  # Alabama
    "GA": 29559,  # Georgia
    "FL": 46405,  # Florida
    "SC": 5680,  # South Carolina
    "NC": 28_348,  # North Carolina
}
S = state_demand  # Sales

print("Total states number: ", len(S))
print("Total demand: ", sum(S.values()))
print("Each state represents a demand center")

# TODO: falta alguém ler e verificar os dados (o input foi feito de forma manual)

# NOTE: Valor total deve ser 1_477_026, então precisa ajustar NY (que nao tem na foto) pra bater o valor final.

Total states number:  48
Total demand:  1477026
Each state represents a demand center


In [4]:
demand_centers = [f"{s}" for s in state_demand.keys()]

pprint(demand_centers)

print(
    """
    The company has more than 70,000 individual customer accounts, and these are aggregated into 191 active demand centers.
    A demand center is a grouping of zip code areas into a zip sectional center as the focus of the collected demand.
    These demand centers, along with how they are currently being served, are given in Table 3
    """
)

['WA',
 'OR',
 'CA',
 'NV',
 'AZ',
 'ID',
 'UT',
 'MT',
 'ND',
 'WY',
 'CO',
 'SD',
 'NE',
 'KS',
 'MN',
 'IA',
 'MO',
 'NM',
 'TX',
 'OK',
 'AR',
 'LA',
 'WI',
 'IL',
 'MI',
 'IN',
 'KY',
 'OH',
 'ME',
 'NH',
 'RI',
 'NJ',
 'NY',
 'PA',
 'CT',
 'MA',
 'VA',
 'WV',
 'MD',
 'VT',
 'DE',
 'TN',
 'MS',
 'AL',
 'GA',
 'FL',
 'SC',
 'NC']

    The company has more than 70,000 individual customer accounts, and these are aggregated into 191 active demand centers.
    A demand center is a grouping of zip code areas into a zip sectional center as the focus of the collected demand.
    These demand centers, along with how they are currently being served, are given in Table 3
    


In [5]:
# 2.2 Plant current capacities C[p]
C = {
    "Covington, KY": 620_000,
    "New York, NY": 430_000,
    "Arlington, TX": 300_000,
    "Long Beach, CA": 280_000,
}

# Plant stocking capacities C'[p]
C_prime = {
    "Covington, KY": 450_000,
    "New York, NY": 380_000,
    "Arlington, TX": 140_000,
    "Long Beach, CA": 180_000,
}

In [6]:
# 2.3 Unit production cost v[p] (variable production cost)
rho = {
    "Covington, KY": 21.0,
    "New York, NY": 19.9,
    "Arlington, TX": 21.6,
    "Long Beach, CA": 21.1,
}

In [7]:
# # 2.4 Distance matrices (in miles)
# #     d_pw[p][w] = distance plant→warehouse
# #     d_wj[w][j] = distance warehouse→demand center
# # For now: mock with Euclidean or random coords or constants
# d_pw = {p: {w: 300.0 for w in warehouses} for p in plants}
# d_wj = {w: {j: 100.0 for j in demand_centers} for w in warehouses}
# d_pd = {p: {d: 100.0 for d in demand_centers} for p in plants}
# # TODO: compute real distances via geopy or similar

# # TODO: ler do arquivo excel diretamente

In [34]:
# 2.4 Distance matrices (in miles)
#     d_pw[p][w] = distance plant→warehouse
#     d_wj[w][j] = distance warehouse→demand center

df_pw = pd.read_excel("distance_matrix.xlsx", sheet_name="d_pw", index_col=0)
df_ws = pd.read_excel("distance_matrix.xlsx", sheet_name="d_ws", index_col=0)
df_pd = pd.read_excel("distance_matrix.xlsx", sheet_name="d_pd", index_col=0)

d_pw = df_pw.to_dict()
d_wj = df_ws.to_dict(orient="index")
d_pd = df_pd.to_dict()

In [35]:
print("Distance matrices:")
print("d_pw (plant to warehouse):")
pprint(d_pw)
print("d_wj (warehouse to demand center):")
pprint(d_wj)
print("d_pd (plant to demand center):")
pprint(d_pd)

Distance matrices:
d_pw (plant to warehouse):
{'Arlington, TX': {'Albuquerque': 570.6765438251061,
                   'Arlington': 0.0,
                   'Atlanta': 739.4459199858047,
                   'Baltimore': 1230.342635327229,
                   'Billings': 1085.304564721067,
                   'Boston': 1568.397345061719,
                   'Buffalo': 1214.304709480124,
                   'Camp Hill': 1233.591807654368,
                   'Chicago': 817.4961795521145,
                   'Chicago *': 817.4961795521145,
                   'Cleveland': 1041.311803777115,
                   'Columbus': 929.6713707311056,
                   'Covington': 830.03403504161,
                   'Davenport': 704.6477550072561,
                   'Denver': 653.0868892115326,
                   'Detroit': 1014.596011842574,
                   'El Paso': 553.107794594599,
                   'Grand Rapids': 941.0454287015159,
                   'Greensboro': 1014.1820115638,
                

In [36]:
# 2.5 Inbound cost c_in[p][w] = 0.92 + 0.0034*d_pw
c_in = {p: {w: 0.92 + 0.0034 * d_pw[p][w] for w in warehouses} for p in plants}

c_in

{'Covington, KY': {'Atlanta': 2.1708526414588722,
  'Boston': 3.4375033043679797,
  'Buffalo': 2.2603078659962463,
  'Chicago': 1.7814791685348303,
  'Cleveland': 1.679310249594197,
  'Davenport': 2.1507544931939395,
  'Detroit': 1.725411401069965,
  'Grand Rapids': 1.853253007200061,
  'Greensboro': 2.048096492022556,
  'Kansas City': 2.768358640429216,
  'Baltimore': 2.3616829257419156,
  'Memphis': 2.309820836797508,
  'Milwaukee': 2.0266748608622667,
  'Orlando': 3.4653702595849354,
  'Pittsburgh': 1.796624582468938,
  'Portland': 7.670786516615866,
  'W Sacramento': 7.658320619804106,
  'W Chester': 2.592900562270025,
  'Albuquerque': 5.1665079450504825,
  'Billings': 5.354799310824214,
  'Denver': 4.641827037491888,
  'El Paso': 5.45810132279653,
  'Camp Hill': 2.3217770957768376,
  'Houston': 3.9527388032647206,
  'Las Vegas': 6.651530421707199,
  'Minneapolis': 2.9821472149394586,
  'New Orleans': 3.315688087560122,
  'Phoenix': 6.294537965757174,
  'Richmond': 2.27573644834448

In [ ]:
# 2.6 Outbound cost c_out[w][j]:
local_rate = {w: 2.0 for w in warehouses}
# TODO: fill from Table 3’s "Local delivery rate" (pegar direto do excel)

c_out = {}
for w in warehouses:
    c_out[w] = {}
    for j in demand_centers:
        d = d_wj[w][j]
        if d <= 30:
            # if d<=30: use local cartage rate from Table 3
            c_out[w][j] = local_rate[w]
        else:
            # else use 5.45 + 0.0037*d
            c_out[w][j] = 5.45 + 0.0037 * d

# c_out

tchau 13.39085830501691
tchau 13.44737647794318
tchau 12.79601627073933
tchau 12.110893228244336
tchau 11.3030242667407
tchau 12.09818484063328
tchau 11.306556824804396
tchau 11.629586092664354
tchau 9.976348063455276
tchau 10.517891878328069
tchau 9.961004996897067
tchau 9.342743377682973
tchau 8.668984303389703
tchau 8.212580529863889
tchau 8.882806648025383
tchau 8.172496993236107
tchau 7.465521685243344
tchau 10.09716382478361
tchau 8.340981436361478
tchau 8.117054859173436
tchau 7.246267945310531
tchau 7.171092419366247
tchau 8.158484782450111
tchau 7.542229997954314
tchau 7.776618103745573
tchau 7.125606264235872
tchau 6.508814444114879
tchau 7.187453970040339
tchau 9.399468259674148
tchau 8.976589893366706
tchau 8.767100609863368
tchau 8.10399119884814
tchau 8.246153250708465
tchau 7.73274929434061
tchau 8.504772894430623
tchau 8.857916297893993
tchau 7.1779354251858525
tchau 6.954661030253553
tchau 7.530436944868368
tchau 8.949619919387038
tchau 7.759295903831271
tchau 6.118354

In [12]:
# TODO: precisa calcular isso daqui.
# custos de transporte da fabrica direto para o demand center.
c_out_prime = {}

for p in plants:
    c_out_prime[p] = {}
    for j in demand_centers:
        try:
            d = d_pd[p][j]
        except KeyError:
            print(f"{p} or {j} not found in d_pd")
            break
        if d <= 30:
            # if d<=30: use local cartage rate from Table 3
            # c_out_prime[p][j] = local_rate[p]
            c_out_prime[p][j] = 0
            
            # TODO: verify this.
        else:
            # else use 5.45 + 0.0037*d
            c_out_prime[p][j] = 5.45 + 0.0037 * d

In [13]:
# Quanto representa 1 cwt. em $? Dividimos as vendas totais ($) pela demand total (cwt)
Gamma = 160_000_000 / 1_477_026  # ($ / cwt)
Gamma

108.32578438023434

In [14]:
tau = {}

# TODO: precisa carregar esse tau direto do excel

for w in warehouses:
    tau[w] = 1.0

tau

{'Atlanta': 1.0,
 'Boston': 1.0,
 'Buffalo': 1.0,
 'Chicago': 1.0,
 'Cleveland': 1.0,
 'Davenport': 1.0,
 'Detroit': 1.0,
 'Grand Rapids': 1.0,
 'Greensboro': 1.0,
 'Kansas City': 1.0,
 'Baltimore': 1.0,
 'Memphis': 1.0,
 'Milwaukee': 1.0,
 'Orlando': 1.0,
 'Pittsburgh': 1.0,
 'Portland': 1.0,
 'W Sacramento': 1.0,
 'W Chester': 1.0,
 'Albuquerque': 1.0,
 'Billings': 1.0,
 'Denver': 1.0,
 'El Paso': 1.0,
 'Camp Hill': 1.0,
 'Houston': 1.0,
 'Las Vegas': 1.0,
 'Minneapolis': 1.0,
 'New Orleans': 1.0,
 'Phoenix': 1.0,
 'Richmond': 1.0,
 'St Louis': 1.0,
 'Salt Lake City': 1.0,
 'San Antonio': 1.0,
 'Seattle': 1.0,
 'Spokane': 1.0,
 'San Francisco': 1.0,
 'Indianapolis': 1.0,
 'Louisville': 1.0,
 'Columbus': 1.0,
 'New York': 1.0,
 'Hartford': 1.0,
 'Miami': 1.0,
 'Mobile': 1.0,
 'Memphis *': 1.0,
 'Chicago *': 1.0}

In [15]:
# 2.7 Warehouse operating cost per cwt: r_w[w]
#     = storage_rate + handling_rate   (from Table 3)
storage = {w: 0.1 for w in warehouses}
handling = {w: 0.7 for w in warehouses}
# TODO: replace storage[w] and handling[w] with Table 3 values


r_w = {w: storage[w] + handling[w] for w in warehouses}

In [16]:
# Big-M for linkage: no warehouse ships more than total demand
# M = sum(S.values())
# M

## 3. Defining the model

In [17]:
m = Model(name="UsemoreWarehousing")

Set parameter Username
Set parameter LicenseID to value 2602866
Academic license - for non-commercial use only - expires 2025-12-22


### Decision variables

In [18]:
Z = m.addVars(warehouses, vtype=GRB.BINARY, name="z")
X = m.addVars(plants, warehouses, vtype=GRB.CONTINUOUS, name="x")
Y = m.addVars(warehouses, demand_centers, vtype=GRB.CONTINUOUS, name="y")
W = m.addVars(plants, demand_centers, vtype=GRB.CONTINUOUS, name="w")

## 4. Objective Function

In [32]:
pprint(c_out)

{'Albuquerque': {'AL': nan,
                 'AR': nan,
                 'AZ': nan,
                 'CA': nan,
                 'CO': nan,
                 'CT': nan,
                 'DE': nan,
                 'FL': nan,
                 'GA': nan,
                 'IA': nan,
                 'ID': nan,
                 'IL': nan,
                 'IN': nan,
                 'KS': nan,
                 'KY': nan,
                 'LA': nan,
                 'MA': nan,
                 'MD': nan,
                 'ME': nan,
                 'MI': nan,
                 'MN': nan,
                 'MO': nan,
                 'MS': nan,
                 'MT': nan,
                 'NC': nan,
                 'ND': nan,
                 'NE': nan,
                 'NH': nan,
                 'NJ': nan,
                 'NM': nan,
                 'NV': nan,
                 'NY': nan,
                 'OH': nan,
                 'OK': nan,
                 'OR': nan,
                 'PA

In [ ]:
pprint(Y)

In [ ]:
m.setObjective(
    # inbound transport (from plants to warehouses)
    gurobipy.quicksum(c_in[i][j] * X[i, j] for i in plants for j in warehouses)
    # outbound transport (from warehouses to demand nodes)
    + gurobipy.quicksum(c_out[j][k] * Y[j, k] for j in warehouses for k in demand_centers)
    # transport cost from plants to demand centers
    + gurobipy.quicksum(c_out_prime[i][k] * W[i, k] for i in plants for k in demand_centers)
    # production cost
    + gurobipy.quicksum(
        rho[p] * (X[p, w] + W[p, d])
        for p in plants
        for w in warehouses
        for d in demand_centers
    )
    # storage cost in warehouses
    + gurobipy.quicksum(X[p, w] * Gamma * tau[w] for p in plants for w in warehouses),
    # TODO: add "custo de handling nos armazéns)"
    # TODO: add ""custo de processamento do pedido de estoque"
    # TODO: add "custo de processamento do pedido do cliente"
    sense=GRB.MINIMIZE,
)

## Constraints

In [20]:
# Demand satisfaction
m.addConstrs((Y.sum("*", d) == S[d] for d in demand_centers), "demand")

{'WA': <gurobi.Constr *Awaiting Model Update*>,
 'OR': <gurobi.Constr *Awaiting Model Update*>,
 'CA': <gurobi.Constr *Awaiting Model Update*>,
 'NV': <gurobi.Constr *Awaiting Model Update*>,
 'AZ': <gurobi.Constr *Awaiting Model Update*>,
 'ID': <gurobi.Constr *Awaiting Model Update*>,
 'UT': <gurobi.Constr *Awaiting Model Update*>,
 'MT': <gurobi.Constr *Awaiting Model Update*>,
 'ND': <gurobi.Constr *Awaiting Model Update*>,
 'WY': <gurobi.Constr *Awaiting Model Update*>,
 'CO': <gurobi.Constr *Awaiting Model Update*>,
 'SD': <gurobi.Constr *Awaiting Model Update*>,
 'NE': <gurobi.Constr *Awaiting Model Update*>,
 'KS': <gurobi.Constr *Awaiting Model Update*>,
 'MN': <gurobi.Constr *Awaiting Model Update*>,
 'IA': <gurobi.Constr *Awaiting Model Update*>,
 'MO': <gurobi.Constr *Awaiting Model Update*>,
 'NM': <gurobi.Constr *Awaiting Model Update*>,
 'TX': <gurobi.Constr *Awaiting Model Update*>,
 'OK': <gurobi.Constr *Awaiting Model Update*>,
 'AR': <gurobi.Constr *Awaiting Model Up

In [21]:
# Plant capacity
m.addConstrs((X.sum(p, "*") <= C[p] for p in plants), "plantCap")

{'Covington, KY': <gurobi.Constr *Awaiting Model Update*>,
 'New York, NY': <gurobi.Constr *Awaiting Model Update*>,
 'Arlington, TX': <gurobi.Constr *Awaiting Model Update*>,
 'Long Beach, CA': <gurobi.Constr *Awaiting Model Update*>}

In [22]:
# TODO: restrição dos 10400

In [23]:
# TODO: restrição do u_{pd}

In [24]:
# Flow balance at each warehouse
# m.addConstrs(( x.sum('*',w) == y.sum(w,'*') for w in warehouses ), "flowBalance")


## Solver

In [25]:
m.params.TimeLimit = 300  # seconds

m.optimize()

Set parameter TimeLimit to value 300
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Arch Linux")

CPU model: Intel(R) Core(TM) i5-10300H CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  300

Optimize a model with 52 rows, 2524 columns and 2288 nonzeros
Model fingerprint: 0xb63fa188
Variable types: 2480 continuous, 44 integer (44 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [9e-01, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+03, 6e+05]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%


In [26]:
m.printStats()

Statistics for model 'UsemoreWarehousing':
  Problem type                : MIP
  Linear constraint matrix    : 52 rows, 2524 columns, 2288 nonzeros
  Variable types              : 2480 continuous, 44 integer (44 binary)
  Matrix range                : [1e+00, 1e+00]
  Objective range             : [9e-01, 1e+01]
  Bounds range                : [1e+00, 1e+00]
  RHS range                   : [1e+03, 6e+05]


In [27]:
m.printQuality()

Solution quality statistics for model 'UsemoreWarehousing' :
  Maximum violation:
    Bound       : 0.00000000e+00
    Constraint  : 0.00000000e+00
    Integrality : 0.00000000e+00


## Visualize results

In [28]:
print("\n--- Warehouse openings ---")
for w in warehouses:
    if Z[w].X > 0.1:
        print(f" Open warehouse at {w}")


--- Warehouse openings ---
 Open warehouse at Atlanta
 Open warehouse at Boston
 Open warehouse at Buffalo
 Open warehouse at Chicago
 Open warehouse at Cleveland
 Open warehouse at Davenport
 Open warehouse at Detroit
 Open warehouse at Grand Rapids
 Open warehouse at Greensboro
 Open warehouse at Kansas City
 Open warehouse at Baltimore
 Open warehouse at Memphis
 Open warehouse at Milwaukee
 Open warehouse at Orlando
 Open warehouse at Pittsburgh
 Open warehouse at Portland
 Open warehouse at W Sacramento
 Open warehouse at W Chester
 Open warehouse at Albuquerque
 Open warehouse at Billings
 Open warehouse at Denver
 Open warehouse at El Paso
 Open warehouse at Camp Hill
 Open warehouse at Houston
 Open warehouse at Las Vegas
 Open warehouse at Minneapolis
 Open warehouse at New Orleans
 Open warehouse at Phoenix
 Open warehouse at Richmond
 Open warehouse at St Louis
 Open warehouse at Salt Lake City
 Open warehouse at San Antonio
 Open warehouse at Seattle
 Open warehouse at Spo

In [29]:
print("\n--- Plant→Warehouse flows ---")
for p, w in X.keys():
    if X[p, w].X > 1e-6:
        print(f" {p} → {w}: {X[p, w].X:.0f} cwt")


--- Plant→Warehouse flows ---


In [30]:
print("\n--- Plant → Demand flows ---")
for w, j in W.keys():
    if W[w, j].X > 1e-6:
        print(f" {w:<13} → {j}: {W[w, j].X:.0f} cwt")


--- Plant → Demand flows ---
 Covington, KY → WA: 100000000 cwt
 Covington, KY → OR: 100000000 cwt
 Covington, KY → CA: 100000000 cwt
 Covington, KY → NV: 100000000 cwt
 Covington, KY → AZ: 100000000 cwt
 Covington, KY → ID: 100000000 cwt
 Covington, KY → UT: 100000000 cwt
 Covington, KY → MT: 100000000 cwt
 Covington, KY → ND: 100000000 cwt
 Covington, KY → WY: 100000000 cwt
 Covington, KY → CO: 100000000 cwt
 Covington, KY → SD: 100000000 cwt
 Covington, KY → NE: 100000000 cwt
 Covington, KY → KS: 100000000 cwt
 Covington, KY → MN: 100000000 cwt
 Covington, KY → IA: 100000000 cwt
 Covington, KY → MO: 100000000 cwt
 Covington, KY → NM: 100000000 cwt
 Covington, KY → TX: 100000000 cwt
 Covington, KY → OK: 100000000 cwt
 Covington, KY → AR: 100000000 cwt
 Covington, KY → LA: 100000000 cwt
 Covington, KY → WI: 100000000 cwt
 Covington, KY → IL: 100000000 cwt
 Covington, KY → MI: 100000000 cwt
 Covington, KY → IN: 100000000 cwt
 Covington, KY → KY: 100000000 cwt
 Covington, KY → OH: 1000

In [31]:
print("\n--- Warehouse→Demand flows ---")
for w, j in Y.keys():
    if Y[w, j].X > 1e-6:
        print(f" {w:<13} → {j}: {Y[w, j].X:.0f} cwt")


--- Warehouse→Demand flows ---
 Atlanta       → WA: 32437 cwt
 Atlanta       → OR: 31365 cwt
 Atlanta       → CA: 135116 cwt
 Atlanta       → NV: 16755 cwt
 Atlanta       → AZ: 9063 cwt
 Atlanta       → ID: 7153 cwt
 Atlanta       → UT: 9001 cwt
 Atlanta       → MT: 4140 cwt
 Atlanta       → ND: 5703 cwt
 Atlanta       → WY: 1004 cwt
 Atlanta       → CO: 11147 cwt
 Atlanta       → SD: 1049 cwt
 Atlanta       → NE: 7347 cwt
 Atlanta       → KS: 6961 cwt
 Atlanta       → MN: 5633 cwt
 Atlanta       → IA: 32175 cwt
 Atlanta       → MO: 41680 cwt
 Atlanta       → NM: 3536 cwt
 Atlanta       → TX: 80438 cwt
 Atlanta       → OK: 13517 cwt
 Atlanta       → AR: 4910 cwt
 Atlanta       → LA: 15011 cwt
 Atlanta       → WI: 37448 cwt
 Atlanta       → IL: 72839 cwt
 Atlanta       → MI: 105181 cwt
 Atlanta       → IN: 43994 cwt
 Atlanta       → KY: 3870 cwt
 Atlanta       → OH: 155123 cwt
 Atlanta       → ME: 15829 cwt
 Atlanta       → NH: 4546 cwt
 Atlanta       → RI: 17000 cwt
 Atlanta       → N